# 14067和icw773R包含基因比较

In [5]:
import pandas as pd
import re
import numpy as np

def clean_string(value):
    """
    清理并验证字符串值，处理空值和非字符串类型
    
    Parameters:
    value: 输入值
    
    Returns:
    str: 处理后的字符串，如果输入无效则返回空字符串
    """
    if pd.isna(value) or value is None:
        return ""
    return str(value).strip()

def extract_gene_mappings(excel_file):
    """
    从Excel文件中提取cgl和cg基因的对应关系，同时从STRING和KEGG列提取
    
    Parameters:
    excel_file (str): Excel文件的路径
    
    Returns:
    pandas.DataFrame: 包含cgl和cg对应关系的数据框
    """
    # 读取Excel文件
    df = pd.read_excel(excel_file)
    
    # 创建用于存储映射关系的列表
    mappings = []
    
    # 遍历每一行数据
    for _, row in df.iterrows():
        try:
            # 清理数据
            kegg_entry = clean_string(row['KEGG'])
            string_entry = clean_string(row['STRING'])
            gene_name = clean_string(row['Gene Names'])
            
            if not any([kegg_entry, string_entry, gene_name]):
                continue
                
            # 从KEGG列提取映射关系
            if kegg_entry:
                # 处理不同格式的KEGG条目
                kegg_matches = []
                # 标准格式：cgb:cg2900;cgl:Cgl2617
                matches1 = re.findall(r'cg[lb]:cg(\d+);cgl:Cgl(\d+)', kegg_entry)
                # 替代格式：cgl:Cgl2668 (没有cg部分)
                matches2 = re.findall(r'cgl:Cgl(\d+)', kegg_entry)
                
                kegg_matches.extend(matches1)
                
                # 如果只找到Cgl编号，尝试从STRING中匹配cg
                if not matches1 and matches2 and string_entry:
                    string_match = re.search(r'cg(\d+)', string_entry)
                    if string_match:
                        for cgl in matches2:
                            kegg_matches.append((string_match.group(1), cgl))
                
                for match in kegg_matches:
                    if len(match) == 2:
                        cg, cgl = match
                        mappings.append({
                            'cg': f'cg{cg}',
                            'cgl': f'Cgl{cgl}',
                            'source': 'KEGG'
                        })
            
            # 如果KEGG中没有找到对应关系，尝试从STRING和Gene Name中提取
            if not kegg_matches and string_entry:
                string_match = re.search(r'cg(\d+)', string_entry)
                cgl_match = re.search(r'Cgl(\d+)', gene_name)
                
                if string_match and cgl_match:
                    mappings.append({
                        'cg': f'cg{string_match.group(1)}',
                        'cgl': f'Cgl{cgl_match.group(1)}',
                        'source': 'STRING'
                    })
                    
        except Exception as e:
            print(f"处理行时出现错误: {str(e)}")
            print(f"问题行数据: {row.to_dict()}")
            continue
    
    # 创建数据框
    result_df = pd.DataFrame(mappings)
    
    if result_df.empty:
        print("警告：没有找到任何映射关系！")
        return pd.DataFrame(columns=['cg', 'cgl', 'source'])
    
    # 去除重复项（保留第一次出现的记录，通常是KEGG的记录）
    result_df = result_df.drop_duplicates(subset=['cg', 'cgl'], keep='first')
    
    # 按cg编号排序
    result_df = result_df.sort_values('cg')
    
    return result_df

def save_mappings(df, output_file):
    """
    将映射结果保存到Excel文件，包含来源信息
    
    Parameters:
    df (pandas.DataFrame): 包含映射关系的数据框
    output_file (str): 输出文件的路径
    """
    if not df.empty:
        # 添加统计信息到控制台
        print("\n映射关系来源统计：")
        print(df['source'].value_counts())
        
        # 保存到Excel
        df.to_excel(output_file, index=False)
        print(f"\n映射关系已保存到 {output_file}")
    else:
        print("没有数据可以保存")

def analyze_coverage(df):
    """
    分析映射覆盖率
    
    Parameters:
    df (pandas.DataFrame): 包含映射关系的数据框
    """
    if df.empty:
        print("\n没有找到任何映射关系，无法分析覆盖率")
        return
        
    total_cg = len(df['cg'].unique())
    total_cgl = len(df['cgl'].unique())
    
    print(f"\n覆盖率统计：")
    print(f"唯一cg基因数量: {total_cg}")
    print(f"唯一cgl基因数量: {total_cgl}")
    print(f"总映射关系数量: {len(df)}")

# 使用示例
if __name__ == "__main__":
    # 假设输入文件名为 "gene_data.xlsx"
    input_file = "8.analysis/uniport of Cg13032.xlsx"
    output_file = "8.analysis/cg_map_cgl.xlsx"
    
    try:
        # 提取映射关系
        mappings_df = extract_gene_mappings(input_file)
        
        # 打印结果预览
        if not mappings_df.empty:
            print("\n前5个映射关系：")
            print(mappings_df.head())
        
        # 分析覆盖率
        analyze_coverage(mappings_df)
        
        # 保存结果
        save_mappings(mappings_df, output_file)
        
    except Exception as e:
        print(f"处理过程中出现错误: {str(e)}")
        print("请检查输入文件格式是否正确")

D:\pycharm\ecmpy2.0\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



前5个映射关系：
          cg      cgl source
402   cg0001  Cgl0001   KEGG
2919  cg0002  Cgl0002   KEGG
870   cg0004  Cgl0003   KEGG
453   cg0005  Cgl0004   KEGG
2918  cg0006  Cgl0005   KEGG

覆盖率统计：
唯一cg基因数量: 2918
唯一cgl基因数量: 2920
总映射关系数量: 2921

映射关系来源统计：
source
KEGG      2920
STRING       1
Name: count, dtype: int64

映射关系已保存到 8.analysis/cg_map_cgl.xlsx


# 提取icw773R的gene信息

In [5]:
import cobra
import pandas as pd
from collections import defaultdict

def extract_gene_info(model_path, output_file):
    """
    从SBML模型中提取基因信息并保存为Excel文件
    
    Parameters:
    model_path (str): SBML模型文件的路径
    output_file (str): 输出Excel文件的路径
    """
    try:
        # 读取SBML模型
        model = cobra.io.read_sbml_model(model_path)
        
        # 创建用于存储基因信息的列表
        gene_info = []
        
        # 创建反应-基因对应关系的字典
        gene_reaction_dict = defaultdict(list)
        
        # 遍历所有反应，建立基因-反应对应关系
        for reaction in model.reactions:
            if reaction.genes:
                for gene in reaction.genes:
                    gene_reaction_dict[gene.id].append(reaction.id)
        
        # 遍历所有基因收集信息
        for gene in model.genes:
            # 获取基因相关的反应
            associated_reactions = gene_reaction_dict[gene.id]
            
            # 创建基因信息字典
            gene_data = {
                'Gene ID': gene.id,
                'Name': gene.name if gene.name else '',
                'Number of Associated Reactions': len(associated_reactions),
                'Associated Reactions': '; '.join(associated_reactions),
                'Gene Formula': str(gene.functional),
            }
            
            # 添加到列表中
            gene_info.append(gene_data)
        
        # 创建DataFrame
        df = pd.DataFrame(gene_info)
        
        # 对基因ID进行排序
        df = df.sort_values('Gene ID')
        
        # 保存到Excel文件
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # 保存基本信息到第一个sheet
            df.to_excel(writer, sheet_name='Gene Information', index=False)
            
            # 创建统计信息sheet
            stats_data = {
                'Metric': [
                    'Total Number of Genes',
                    'Genes with Associated Reactions',
                    'Average Reactions per Gene',
                    'Max Reactions per Gene',
                    'Min Reactions per Gene'
                ],
                'Value': [
                    len(df),
                    len(df[df['Number of Associated Reactions'] > 0]),
                    df['Number of Associated Reactions'].mean(),
                    df['Number of Associated Reactions'].max(),
                    df['Number of Associated Reactions'].min()
                ]
            }
            stats_df = pd.DataFrame(stats_data)
            stats_df.to_excel(writer, sheet_name='Statistics', index=False)
        
        print(f"成功提取基因信息并保存到 {output_file}")
        print(f"\n基本统计信息：")
        print(f"总基因数：{len(df)}")
        print(f"有关联反应的基因数：{len(df[df['Number of Associated Reactions'] > 0])}")
        print(f"每个基因平均关联反应数：{df['Number of Associated Reactions'].mean():.2f}")
        
        return df
        
    except Exception as e:
        print(f"处理过程中出现错误: {str(e)}")
        raise

def analyze_gene_associations(df):
    """
    分析基因与反应的关联关系
    
    Parameters:
    df (pandas.DataFrame): 包含基因信息的数据框
    """
    # 分析基因关联反应数的分布
    reaction_counts = df['Number of Associated Reactions'].value_counts().sort_index()
    
    print("\n基因关联反应数分布：")
    for count, num_genes in reaction_counts.items():
        print(f"关联 {count} 个反应的基因数量：{num_genes}")
    
    # 找出关联反应数最多的基因
    top_genes = df.nlargest(5, 'Number of Associated Reactions')
    print("\n关联反应数最多的前5个基因：")
    for _, gene in top_genes.iterrows():
        print(f"基因 {gene['Gene ID']}: {gene['Number of Associated Reactions']} 个反应")

# 使用示例
if __name__ == "__main__":
    # 设置输入输出文件路径
    model_file = "other model/iCW773R.xml"  # 替换为您的模型文件路径
    output_file = "8.analysis/icw773R_genes.xlsx"
    
    try:
        # 提取基因信息
        gene_df = extract_gene_info(model_file, output_file)
        
        # 分析基因关联关系
        analyze_gene_associations(gene_df)
        
    except Exception as e:
        print(f"程序执行失败: {str(e)}")
        print("请检查输入文件路径是否正确，以及是否安装了所需的库")

'' is not a valid SBML 'SId'.


成功提取基因信息并保存到 8.analysis/icw773R_genes.xlsx

基本统计信息：
总基因数：786
有关联反应的基因数：744
每个基因平均关联反应数：2.04

基因关联反应数分布：
关联 0 个反应的基因数量：42
关联 1 个反应的基因数量：446
关联 2 个反应的基因数量：187
关联 3 个反应的基因数量：33
关联 4 个反应的基因数量：29
关联 5 个反应的基因数量：6
关联 6 个反应的基因数量：2
关联 7 个反应的基因数量：13
关联 8 个反应的基因数量：1
关联 9 个反应的基因数量：1
关联 10 个反应的基因数量：3
关联 11 个反应的基因数量：1
关联 12 个反应的基因数量：1
关联 14 个反应的基因数量：5
关联 15 个反应的基因数量：7
关联 16 个反应的基因数量：1
关联 17 个反应的基因数量：1
关联 20 个反应的基因数量：2
关联 21 个反应的基因数量：1
关联 23 个反应的基因数量：3
关联 24 个反应的基因数量：1

关联反应数最多的前5个基因：
基因 Cgl0719: 24 个反应
基因 Cgl2491: 23 个反应
基因 Cgl2683: 23 个反应
基因 Cgl2872: 23 个反应
基因 Cgl2254: 21 个反应


# 比较模型之间的gene差异

In [1]:
import pandas as pd
import numpy as np
import cobra

def get_gene_reactions_from_xml(model_path):
    """
    从XML模型中提取基因-反应对应关系，只返回反应ID
    """
    # 读取SBML模型
    model = cobra.io.read_sbml_model(model_path)
    
    # 创建基因-反应映射字典
    gene_to_reactions = {}
    
    # 遍历模型中的反应
    for reaction in model.reactions:
        if reaction.genes:  # 如果反应有关联的基因
            # 只保留反应ID
            reaction_str = reaction.id
            
            # 为每个基因添加这个反应
            for gene in reaction.genes:
                if gene.id not in gene_to_reactions:
                    gene_to_reactions[gene.id] = []
                gene_to_reactions[gene.id].append(reaction_str)
    
    return gene_to_reactions

def compare_model_genes(cey_file, icw_file, mapping_file, uniprot_file, xml_model_path, output_file):
    """
    比较两个模型的基因异同
    """
    try:
        # 读取文件
        cey_df = pd.read_excel(cey_file, sheet_name='genes')
        icw_df = pd.read_excel(icw_file)
        mapping_df = pd.read_excel(mapping_file)
        uniprot_df = pd.read_excel(uniprot_file)
        
        # 从XML模型中获取基因-反应对应关系
        gene_reactions = get_gene_reactions_from_xml(xml_model_path)
        
        # 创建CGL到CG的映射字典
        cgl_to_cg = dict(zip(mapping_df['cgl'], mapping_df['cg']))
        
        # 提取UniProt中的基因和蛋白信息
        protein_info = {}
        for _, row in uniprot_df.iterrows():
            kegg = str(row['KEGG'])
            if pd.notna(kegg) and 'cgl:' in kegg.lower():
                cgl_numbers = [part.split(':')[1].strip() for part in kegg.split(';') 
                             if 'cgl:' in part.lower()]
                for cgl in cgl_numbers:
                    protein_info[cgl] = {
                        'Entry': row['Entry'],
                        'Protein_names': row['Protein names'] if 'Protein names' in row.index else '',
                        'Gene_Name': row['Gene Name'] if 'Gene Name' in row.index else '',
                        'Length': row['Length'] if 'Length' in row.index else '',
                        'Organism': row['Organism'] if 'Organism' in row.index else ''
                    }
        
        # 确保mapping_df中有正确的列名
        if 'cgl' in mapping_df.columns:
            mapping_column = 'cgl'
        elif 'cgL' in mapping_df.columns:
            mapping_column = 'cgL'
        else:
            raise ValueError("Mapping文件中没有找到'cgl'或'cgL'列")
            
        # 使用正确的列名创建CEY基因数据框
        cey_genes = cey_df[['GeneID', 'Gene_name', 'Predicted_Genes', 'Product']]
        
        # 通过mapping关系，建立CEY基因和cgl的对应
        merged_df = pd.merge(cey_genes, mapping_df, 
                           left_on='Predicted_Genes', 
                           right_on='cg', 
                           how='left')
        
        # 将icw模型的基因信息整理为字典格式
        icw_genes_dict = {gene: reactions for gene, reactions in 
                         zip(icw_df['Gene ID'], icw_df['Associated Reactions'])}
        
        # 创建结果列表
        comparison_results = []
        
        # 遍历合并后的数据
        for _, row in merged_df.iterrows():
            cey_id = row['GeneID']
            cgl_id = row[mapping_column] if pd.notna(row[mapping_column]) else None
            gene_name = row['Gene_name']
            product = row['Product']
            
            # 获取XML模型中的反应，用分号连接
            xml_reactions = '; '.join(gene_reactions.get(cey_id, []))
            
            # 检查基因在icw模型中是否存在
            icw_reactions = icw_genes_dict.get(cgl_id, None) if cgl_id else None
            
            status = 'Only in CEY' if not cgl_id or cgl_id not in icw_genes_dict else 'Common'
            
            comparison_results.append({
                'CEY_Gene_ID': cey_id,
                'Gene_Name': gene_name,
                'Product': product,
                'CGL_ID': cgl_id,
                'CG_ID': row['Predicted_Genes'],
                'Status': status,
                'CEY_Reactions': xml_reactions,
                'ICW_Reactions': icw_reactions if icw_reactions else ''
            })
        
        # 检查ICW特有的基因并添加蛋白信息
        for icw_gene, reactions in icw_genes_dict.items():
            if icw_gene not in merged_df[mapping_column].values:
                # 获取蛋白质信息
                protein_data = protein_info.get(icw_gene, {})
                
                comparison_results.append({
                    'CEY_Gene_ID': '',
                    'Gene_Name': protein_data.get('Gene_Name', ''),
                    'Product': protein_data.get('Protein_names', ''),
                    'CGL_ID': icw_gene,
                    'CG_ID': cgl_to_cg.get(icw_gene, ''),
                    'Status': 'Only in ICW',
                    'CEY_Reactions': '',
                    'ICW_Reactions': reactions,
                    'UniProt_Entry': protein_data.get('Entry', ''),
                    'Protein_Length': protein_data.get('Length', ''),
                    'Organism': protein_data.get('Organism', '')
                })
        
        # 创建结果DataFrame
        result_df = pd.DataFrame(comparison_results)
        
        # 计算统计信息
        total_cey = len(cey_genes)
        total_icw = len(icw_genes_dict)
        common_genes = len(result_df[result_df['Status'] == 'Common'])
        only_cey = len(result_df[result_df['Status'] == 'Only in CEY'])
        only_icw = len(result_df[result_df['Status'] == 'Only in ICW'])
        
        # 保存结果到Excel文件
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # 保存详细比较结果
            result_df.to_excel(writer, sheet_name='Gene Comparison', index=False)
            
            # 创建统计信息sheet
            stats_data = {
                'Metric': [
                    'Total CEY Genes',
                    'Total ICW Genes',
                    'Common Genes',
                    'Only in CEY',
                    'Only in ICW',
                    'CEY Coverage (%)',
                    'ICW Coverage (%)'
                ],
                'Value': [
                    total_cey,
                    total_icw,
                    common_genes,
                    only_cey,
                    only_icw,
                    round(common_genes/total_cey * 100, 2),
                    round(common_genes/total_icw * 100, 2)
                ]
            }
            stats_df = pd.DataFrame(stats_data)
            stats_df.to_excel(writer, sheet_name='Statistics', index=False)
            
            # 创建三个子集的sheet
            common_df = result_df[result_df['Status'] == 'Common']
            cey_only_df = result_df[result_df['Status'] == 'Only in CEY']
            icw_only_df = result_df[result_df['Status'] == 'Only in ICW']
            
            common_df.to_excel(writer, sheet_name='Common Genes', index=False)
            cey_only_df.to_excel(writer, sheet_name='CEY Only', index=False)
            icw_only_df.to_excel(writer, sheet_name='ICW Only', index=False)
        
        print(f"\n基因比较统计信息：")
        print(f"CEY模型总基因数：{total_cey}")
        print(f"ICW模型总基因数：{total_icw}")
        print(f"共同基因数：{common_genes}")
        print(f"CEY特有基因数：{only_cey}")
        print(f"ICW特有基因数：{only_icw}")
        print(f"CEY基因覆盖率：{round(common_genes/total_cey * 100, 2)}%")
        print(f"ICW基因覆盖率：{round(common_genes/total_icw * 100, 2)}%")
        
        return result_df
        
    except Exception as e:
        print(f"处理过程中出现错误: {str(e)}")
        import traceback
        print(traceback.format_exc())
        raise

# 使用示例
if __name__ == "__main__":
    cey_file = "14067gem20_CGXII.xlsx"
    icw_file = "8.analysis/icw773R_genes.xlsx"
    mapping_file = "8.analysis/cg_map_cgl.xlsx"
    uniprot_file = "8.analysis/uniport of Cg13032.xlsx"
    xml_model_path = "14067gem20_CGXII.xml"
    output_file = "8.analysis/14067vsiCW773R.xlsx"
    
    try:
        comparison_results = compare_model_genes(cey_file, icw_file, mapping_file, 
                                               uniprot_file, xml_model_path, output_file)
        print(f"\n比较结果已保存到 {output_file}")
        
    except Exception as e:
        print(f"程序执行失败: {str(e)}")
        print("请检查输入文件是否存在且格式正确")

D:\pycharm\ecCG827\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


No objective coefficients in model. Unclear what should be optimized



基因比较统计信息：
CEY模型总基因数：844
ICW模型总基因数：786
共同基因数：548
CEY特有基因数：296
ICW特有基因数：238
CEY基因覆盖率：64.93%
ICW基因覆盖率：69.72%

比较结果已保存到 8.analysis/14067vsiCW773R.xlsx
